In [1]:
import logging

logger = logging.getLogger()
logger.handlers.clear()

# format
formatter = logging.Formatter('[%(asctime)s] [%(levelname)s] %(message)s')

# add stream handler
handler = logging.StreamHandler(sys.stdout)                             
handler.setFormatter(formatter)
logger.addHandler(handler)

# enable all logging level
logger.setLevel(logging.WARNING)

Definition for the board.

In [2]:
import os

os.environ["OKFP_ROOT"] = "C:/Users/sean9/Downloads/FrontPanelUSB"

from ece437.ok import OKFrontPanel

path = "C:/Users/sean9/ECE437/Final/Final.runs/impl_1/final_top.bit"
fp = OKFrontPanel(firmware_path=path)

Start control DRV8833 over PMOD port 1.

In [3]:
from ece437.presets import I2C_1_ENDPOINTS, PMOD_1_ENDPOINTS
from ece437.i2c import I2CController
from ece437.sensor import LSM303Accelerometer, LSM303Magnetometer
from ece437.pmod import PMOD, PMODType5Endpoints, PMODDirection
import time
import numpy as np

with fp:
    logger.info(f"serial={fp.serial_number}")

    with I2CController(fp, I2C_1_ENDPOINTS) as i2c1:
        acc = LSM303Accelerometer(i2c1)
        acc.configure() # defaults

        mag = LSM303Magnetometer(i2c1)
        mag.configure() # defaults

        n_measurements = 100
        n_repeats = 10

        acc_max_hist = []

        for i in range(n_repeats):
            logger.warning(f'trial {i+1}')
            acc_hist = []

            with PMOD(fp, PMOD_1_ENDPOINTS) as pmod:
                logger.info('reset')
                pmod.start(2000, PMODDirection.Forward, block=True)

                logger.info('start moving')
                pmod.start(200, PMODDirection.Reverse)
                logger.info('start recording')
                for i in range(n_measurements):
                    acc_hist.append(acc.read())

            acc_hist = np.array(acc_hist)

            acc_max = abs(acc_hist).max(axis=0)
            acc_max_hist.append(acc_max)
        
        acc_max_hist = np.array(acc_max_hist)
        print(f'avg={acc_max_hist.mean(axis=0)}')
        print(f'std={acc_max_hist.std(axis=0)}')
        

[2022-12-15 15:13:27,275] [WARNING] trial 1
[2022-12-15 15:13:29,673] [WARNING] trial 2
[2022-12-15 15:13:30,273] [WARNING] trial 3
[2022-12-15 15:13:30,868] [WARNING] trial 4
[2022-12-15 15:13:31,458] [WARNING] trial 5
[2022-12-15 15:13:32,053] [WARNING] trial 6
[2022-12-15 15:13:32,649] [WARNING] trial 7
[2022-12-15 15:13:33,244] [WARNING] trial 8
[2022-12-15 15:13:33,837] [WARNING] trial 9
[2022-12-15 15:13:34,428] [WARNING] trial 10
avg=[1.43   0.3096 0.8324]
std=[0.1419211  0.1163058  0.28830234]
